frequency가 2 이상인 데이터 추출 순서
1. frequency가 2 이상인 단어 제외하고 다 삭제
2. hop_num 다시 계산한다. (null인 경우 0으로 처리)
3. 각 단어가 20개 이상인 데이터만 남겨두고 삭제


In [1]:
import pandas as pd
import json
import random

df = pd.read_excel("../../data/one_hop/preprocessed_df2.xlsx")
df.fillna('null',inplace=True)
dx = df.copy()

with open('../../data/one_hop/word_frequency.json', "r") as f:
    word_counts = json.load(f)

In [2]:
df.head()

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,view
0,0,Danielle Darrieux,Q234149,"Georges Mitsinkides,France,European Film Award...",21,French,Q150,"Niger,past imperative,past anterior,language,I...",99,English,Q1860,"Eritrea,religious denomination,Fiji,Sri Lanka,...",175,222931
1,1,Edwin of Northumbria,Q348955,"English,Whitby Abbey,Hatfield Chase,Catholic s...",12,Christianity,Q5043,"worldwide,Jerusalem,Bible,Christendom,Christia...",25,Islam,Q432,"worldwide,New International Encyclopedia,human...",76,177019
2,2,Toko Yasuda,Q7813654,"keyboard instrument,bass guitar,Enon,bass guit...",13,guitar,Q6607,"plucked necked box lute,fretboard,gitarre,viol...",32,piano,Q5994,"keyboard instrument,Metropolitan Museum of Art...",11,74330
3,3,Autonomous University of Madrid,Q788091,"YERUN,UAM,European Alliance for Social Science...",9,Spain,Q29,"Turkey,Russia,African Development Bank,spaniol...",231,Sweden,Q34,"SE,Scania,African Development Bank,Turkey,Russ...",212,87615
4,4,Lyon,Q456,"Guangzhou,Gothenburg Municipality,NADD Wikidat...",47,Beirut,Q3820,"Saifi,Dar Mreisseh,Buenos Aires,Bachoura,Mazra...",51,Manila,Q1461,"Pasig River,Winnipeg,San Francisco,Queen City ...",93,2376138


In [3]:
for i in range(len(df)):
    if (df.loc[i, 'sbj_one_hop'] == 'null'):
        print(df.loc[i, 'index'],df.loc[i, 'subject'],",", df.loc[i, 'subject_id'], ",", df.loc[i, 'obj_true'],",", df.loc[i, 'obj_new'])

125 Manila Light Rail Transit System , Q4165607 , Philippines , Ukraine
288 Massenet Festival , Q2901924 , France , Albania
354 Caribbean Community Secretariat , Q7444221 , Georgetown , Frankfurt
710 Taiwan Pride , Q7676561 , Taiwan , Manchester
782 Honda CX series , Q1626555 , Honda , Airbus
974 Chu Lai Base Area , Q18639545 , Vietnam , Australia
1213 GNU/Linux naming controversy , Q1423906 , Linux , Minnesota
1236 Squad 51 , Q7581888 , Dodge , Suzuki
1281 Al Daayen , Q310786 , Qatar , Iraq
1311 Ma'an Governorate , Q606340 , Jordan , Naples
1484 Battle of the Menin Road Ridge , Q4872951 , Belgium , Atlanta
1492 community development block in India , Q2775236 , India , Romania
1499 Turkey national rugby union team , Q1634826 , Turkey , Romania
1625 Yokohama International School , Q8054631 , Japan , Netherlands
1661 Buddhist philosophy , Q1001079 , Buddhism , Christianity
1748 Kranji War Memorial , Q2945352 , Singapore , Jerusalem
1972 Clan Cumming , Q1094924 , Scotland , Shanghai
2049 

In [4]:
def filtering(word, word_counts, min_count=2):
    if word == 'null':
        return 0, 'null'
    word_list = word.split(',')
    word_list = [word for word in word_list if word_counts.get(word, 0) >= min_count]
    word_list = sorted(word_list, key=lambda x: word_counts.get(x, 0), reverse=True)
    num = len(word_list)
    return num, ','.join(word_list)

def filtering2(dx, min_count, k):
    df = dx.copy()
    for i in range(len(df)):
        df.loc[i, 'sbj_hop_num'], df.loc[i, 'sbj_one_hop'] = filtering(df.loc[i, 'sbj_one_hop'], word_counts, min_count)
        df.loc[i, 'obj_true_hop_num'], df.loc[i, 'obj_one_hop'] = filtering(df.loc[i, 'obj_one_hop'], word_counts, min_count)
        df.loc[i, 'obj_new_hop_num'], df.loc[i, 'obj_new_one_hop'] = filtering(df.loc[i, 'obj_new_one_hop'], word_counts, min_count)

    well_list = []

    for i in range(len(df)):
        if (df.loc[i, 'sbj_hop_num']>=k and df.loc[i, 'obj_true_hop_num']>=k and df.loc[i, 'obj_new_hop_num']>=k):
            well_list.append(i)
    print(f'Total number of data with (freq >= {min_count}) and (length >= {k}) is {len(well_list)}({round(100 * len(well_list) / 21919, 2)}%)')
    

In [ ]:
df = dx.copy()
for min_count in range(1, 11):
    for k in range(0, 11):
        filtering2(df, min_count, k)

Total number of data with (freq >= 1) and (length >= 0) is 21919(100.0%)
Total number of data with (freq >= 1) and (length >= 1) is 21781(99.37%)
Total number of data with (freq >= 1) and (length >= 2) is 21014(95.87%)
Total number of data with (freq >= 1) and (length >= 3) is 19435(88.67%)
Total number of data with (freq >= 1) and (length >= 4) is 18077(82.47%)
Total number of data with (freq >= 1) and (length >= 5) is 16377(74.72%)
Total number of data with (freq >= 1) and (length >= 6) is 14983(68.36%)
Total number of data with (freq >= 1) and (length >= 7) is 13861(63.24%)
Total number of data with (freq >= 1) and (length >= 8) is 12370(56.44%)
Total number of data with (freq >= 1) and (length >= 9) is 11284(51.48%)
Total number of data with (freq >= 1) and (length >= 10) is 10322(47.09%)
Total number of data with (freq >= 2) and (length >= 0) is 21919(100.0%)
Total number of data with (freq >= 2) and (length >= 1) is 21591(98.5%)
Total number of data with (freq >= 2) and (length >

In [ ]:
sample_list = random.sample(well_list, 1000)
sample_list.sort()
df = df.iloc[sample_list]


In [ ]:
df = df.reset_index(drop = True)
df

,index,subject,subject_id,sbj_one_hop,sbj_hop_num,obj_true,obj_true_id,obj_one_hop,obj_true_hop_num,obj_new,obj_new_id,obj_new_one_hop,obj_new_hop_num,view
0,4,Lyon,Q456,"Europe,France,Kyiv,NADD Wikidata project,Saint...",37,Beirut,Q3820,"Catholic Encyclopedia,Lebanon,largest city,Cre...",39,Manila,Q1461,"New International Encyclopedia,Philippines,Asi...",82,2376138
1,90,Rabat,Q3551,"Granat Encyclopedic Dictionary,Armenian Soviet...",20,Damascus,Q3766,"Catholic Encyclopedia,Bible Encyclopedia of Ar...",37,Istanbul,Q406,"Europe,Sytin Military Encyclopedia,Asia,Creati...",70,1201331
2,143,Delusions of Grandeur,Q705247,"Germany,Spain,Madrid,Barcelona,film,color,vide...",21,French,Q150,"Belgium,Switzerland,Algeria,Vietnam,Morocco,La...",71,English,Q1860,"Granat Encyclopedic Dictionary,Armenian Soviet...",149,28776
3,145,Falling Down,Q289598,"Los Angeles,film,color,video on demand,drama f...",33,France,Q142,"Jewish Encyclopedia of Brockhaus and Efron,Eur...",185,Philippines,Q928,"New International Encyclopedia,Laos,Sri Lanka,...",275,2256340
4,168,Cees Nooteboom,Q158394,"human,male,writer,works protected by copyright...",32,Dutch,Q7411,"New International Encyclopedia,Small Brockhaus...",53,Indonesian,Q9240,"English,Indonesia,Arabic,agglutinative languag...",28,43963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,21872,Paul Huet,Q1354027,"human,male,National Gallery of Victoria,copyri...",33,Paris,Q90,"Catholic Encyclopedia,Europe,Genadendal Music ...",60,Venice,Q641,"Granat Encyclopedic Dictionary,Italy,big city,...",78,10248
996,21893,Wilkie Collins,Q210740,"human,male,writer,United Kingdom of Great Brit...",22,London,Q84,"Granat Encyclopedic Dictionary,Catholic Encycl...",104,Chicago,Q1297,"United States of America,New International Enc...",25,397478
997,21907,Mary Wollstonecraft,Q101638,"human,New International Encyclopedia,German,Lo...",42,French,Q150,"Italy,Belgium,Switzerland,Algeria,Vietnam,Moro...",73,English,Q1860,"Granat Encyclopedic Dictionary,Armenian Soviet...",147,1873516
998,21909,Tel Aviv,Q33935,"English,Israel,Mexico City,Organization of Wor...",42,Milan,Q490,"Granat Encyclopedic Dictionary,Jewish Encyclop...",65,Tokyo,Q1490,"Asia,largest city,Japan,Berlin,megacity,global...",89,4019548
